In [11]:
#1. Using MNIST dataset, consider the following procedure. 

In [12]:
import sklearn.datasets as dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

mnist = dataset.fetch_openml('mnist_784')
print(type(mnist.data))

mnist_data = mnist.data.to_numpy()
mnist_target = mnist.target.to_numpy()
print('mnist data shape : ',mnist_data.shape)
print('mnist target shape : ',mnist_target.shape)

<class 'pandas.core.frame.DataFrame'>
mnist data shape :  (70000, 784)
mnist target shape :  (70000,)


In [13]:
#A. Randomly select 10,000 images for dataset. 

print(mnist_target)

binary_idx = np.where( mnist_target < '2')
Ternary_idx = np.where( mnist_target < '3')
print(len(binary_idx[0]))
print(len(Ternary_idx[0]))

['5' '0' '4' ... '4' '5' '6']
14780
21770


In [14]:
def select(data, target, number=10000):
    subset_idx = np.sort( np.random.choice(len(data), number,replace=False) )
    
    subset_data = data[subset_idx]
    subset_target = target[subset_idx]
    return subset_data, subset_target

binary_data, binary_target  = select10000(mnist_data[binary_idx], mnist_target[binary_idx])
ternary_data, ternary_target = select10000(mnist_data[Ternary_idx], mnist_target[Ternary_idx])

In [15]:
#B. Separate the dataset into training and test set. 

def separate(data,target,testset_rate=0.2):
    rate = testset_rate

    testset_idx = np.sort( np.random.choice(len(data), round( len(data)* rate),replace=False) )
    trainingset_idx = np.sort( np.setdiff1d(np.arange(len(data)), testset_idx) )

    #print('testset_idx \n')
    #print(testset_idx)
    #print('\n trainingset_idx \n')
    #print(trainingset_idx)

    test_data = data[testset_idx]
    test_target = target[testset_idx]

    training_data = data[trainingset_idx]
    training_target = target[trainingset_idx]

    #print('\n test set shape:  ', test_data.shape, test_target.shape )
    #print('\n training set shape:  ', training_data.shape, training_target.shape )
    
    return test_data, test_target, training_data,training_target

binary_test_data, binary_test_target, binary_training_data, binary_training_target = separate(binary_data, binary_target)
print()
ternary_test_data, ternary_test_target, ternary_training_data, ternary_training_target = separate(ternary_data, ternary_target)

In [16]:
#C. Train the data using SVM library. 
#D. Test the trained parameters using test set. 
from sklearn import svm
binary = svm.SVC(kernel='linear',decision_function_shape='ovo')
binary.fit(binary_training_data, binary_training_target)
print('training set accuracy = %.2f' % binary.score(binary_training_data, binary_training_target))
print()
print('Test set')
predict = binary.predict(binary_test_data)
count = np.sum (predict == binary_test_target)
print('test data lenght: ', len(binary_test_data), 'correct count: ', count)
print('accuracy: ', 100*(count/len(binary_test_data)), '%')

training set accuracy = 1.00

Test set
test data lenght:  2000 correct count:  1998
accuracy:  99.9 %


In [18]:
ternary = svm.SVC(kernel='linear',decision_function_shape='ovr')
ternary.fit(ternary_training_data, ternary_training_target)
print('training set accuracy = %.2f' % ternary.score(ternary_training_data, ternary_training_target))
print()
print('Test set')
predict = ternary.predict(ternary_test_data)
count = np.sum (predict == ternary_test_target)
print('test data lenght: ', len(ternary_test_target), 'correct count: ', count)
print('accuracy: ', 100*(count/len(ternary_test_target)), '%')

training set accuracy = 1.00

Test set
test data lenght:  2000 correct count:  1963
accuracy:  98.15 %


In [21]:
#E. Repeat step B-D using k-fold (k=4) cross-validation method.

subset_data, subset_target = select(mnist_data,mnist_target,20000)

model = svm.SVC(kernel='linear',decision_function_shape='ovr')
for i in range(5):
    validation_test_data, validation_test_target,ternary_training_data, ternary_training_target = separate(subset_data, subset_target, testset_rate=0.25)
    model.fit(ternary_training_data, ternary_training_target)
    print('epoch', i ,'training set accuracy = %.2f' % model.score(ternary_training_data, ternary_training_target))
    predict = ternary.predict(validation_test_data)
    print('epoch', i ,'validation set accuracy = %.2f' % model.score(validation_test_data, validation_test_target))
    print()

epoch 0 training set accuracy = 1.00
epoch 0 validation set accuracy = 0.90

epoch 1 training set accuracy = 1.00
epoch 1 validation set accuracy = 0.90

epoch 2 training set accuracy = 1.00
epoch 2 validation set accuracy = 0.89

epoch 3 training set accuracy = 1.00
epoch 3 validation set accuracy = 0.91

epoch 4 training set accuracy = 1.00
epoch 4 validation set accuracy = 0.90

